In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
t = 0

In [3]:
if t == 1:
    df_t_9 = pd.read_csv("test9.csv", index_col='id')
else:
    df_t_9 = pd.read_csv("train9.csv", index_col='id')

In [4]:
df_t_9.head()

,user_id,hero_id,num_games,num_wins,kda_ratio
id,,,,,
1_74,1,74,304,185,3743.699732
1_65,1,65,255,151,4009.021513
1_39,1,39,203,135,5094.621514
1_106,1,106,158,106,4619.212963
1_2,1,2,157,94,2871.627907


In [5]:
if t == 1:
    df_t_1 = pd.read_csv("test1.csv", index_col='id')
else:
    df_t_1 = pd.read_csv("train1.csv", index_col='id')

In [6]:
df_t_1.head()

,user_id,hero_id,num_games,num_wins,kda_ratio
id,,,,,
1_41,1,41,201,125,3731.707317
2_17,2,17,427,234,2922.058381
5_106,5,106,240,130,5116.682739
6_65,6,65,206,122,4131.553861
7_69,7,69,697,382,2474.469306


In [7]:
df_hero = pd.read_csv("hero_data.csv")

In [8]:
df_t_9_merged = pd.merge(right=df_hero, left=df_t_9, on='hero_id', how='inner', sort=False)

In [9]:
df_t_1_merged = pd.merge(right=df_hero, left=df_t_1, on='hero_id', how='inner', sort=False)

In [10]:
df_t_9_num = df_t_9_merged.drop(['user_id','hero_id','primary_attr','attack_type', 'roles','kda_ratio','num_wins'], axis=1 )

In [11]:
df_t_1_num = df_t_1_merged.drop(['user_id','hero_id','primary_attr','attack_type', 'roles','kda_ratio','num_wins'], axis=1 )

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaler = MinMaxScaler()

In [14]:
scaled_df_t_9_num = scaler.fit_transform(df_t_9_num)
scaled_df_t_1_num = scaler.transform(df_t_1_num)

In [15]:
y_train = df_t_9_merged['kda_ratio']
y_test = df_t_1_merged['kda_ratio']

In [16]:
df_scaled_df_t_9_num = pd.DataFrame(data=scaled_df_t_9_num,columns = df_t_9_num.columns,index=df_t_9_num.index)

In [17]:
df_scaled_df_t_1_num = pd.DataFrame(data=scaled_df_t_1_num,columns = df_t_1_num.columns,index=df_t_1_num.index)

In [18]:
X_train = pd.concat([df_t_9_merged[['user_id','hero_id','primary_attr','attack_type', 'roles']], df_scaled_df_t_9_num], axis=1)

In [19]:
X_test = pd.concat([df_t_1_merged[['user_id','hero_id','primary_attr','attack_type', 'roles']], df_scaled_df_t_1_num], axis=1)

In [20]:
import tensorflow as tf

In [21]:
primary_attr = tf.feature_column.categorical_column_with_vocabulary_list("primary_attr", ["int", "agi", "str"])
attack_type = tf.feature_column.categorical_column_with_vocabulary_list("attack_type", ["Ranged", "Melee"])
roles = tf.feature_column.categorical_column_with_hash_bucket("roles", hash_bucket_size=1000)
hero_id = tf.feature_column.categorical_column_with_identity("hero_id", num_buckets=150)
user_id = tf.feature_column.categorical_column_with_identity("user_id", num_buckets=5000)

In [22]:
primary_attr_e = tf.feature_column.embedding_column(primary_attr, dimension=3)
attack_type_e = tf.feature_column.embedding_column(attack_type, dimension=2)
roles_e = tf.feature_column.embedding_column(roles, dimension=1000)
hero_id_e = tf.feature_column.embedding_column(hero_id, dimension=150)
user_id_e = tf.feature_column.embedding_column(user_id, dimension=5000)

In [23]:
feat_col = []

for col in df_t_9_num.columns:
    feat_col.append(tf.feature_column.numeric_column(col))

In [24]:
feat_col.append(primary_attr_e)
feat_col.append(attack_type_e)
feat_col.append(hero_id_e)
feat_col.append(roles_e)
feat_col.append(user_id_e)

In [81]:
input_fn = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, shuffle=True, num_epochs=10, batch_size=100)

In [82]:
model = tf.estimator.DNNRegressor(hidden_units=[6], 
                                  feature_columns=feat_col
                                  
                                )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\RANBEE~1\\AppData\\Local\\Temp\\tmpnmp6wq30', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001DE11D11A90>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [83]:
model.train(input_fn=input_fn, steps=4000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\RANBEE~1\AppData\Local\Temp\tmpnmp6wq30\model.ckpt.
INFO:tensorflow:loss = 1.17538e+09, step = 1
INFO:tensorflow:global_step/sec: 24.783
INFO:tensorflow:loss = 1.19267e+08, step = 101 (4.039 sec)
INFO:tensorflow:global_step/sec: 27.5818
INFO:tensorflow:loss = 7.82882e+07, step = 201 (3.637 sec)
INFO:tensorflow:global_step/sec: 26.0747
INFO:tensorflow:loss = 4.90058e+07, step = 301 (3.821 sec)
INFO:tensorflow:global_step/sec: 28.7355
INFO:tensorflow:loss = 5.66839e+07, step = 401 (3.483 sec)
INFO:tensorflow:global_step/sec: 26.68
INFO:tensorflow:loss = 4.59795e+07, step = 501 (3.750 sec)
INFO:tensorflow:global_step/sec: 27.1354
INFO:tensorflow:loss = 5.17421e+07, step = 601 (3.679 sec)
INFO:tensorflow:global_step/sec: 27.6341
INFO:tensorflow:loss = 5.1971e+07, step = 701 (3.623 sec)
INFO:tensorflow:global_step/sec: 27.1786
INFO:tensorflow:loss = 3.39426e+07, step = 801 (3.683 sec)
INFO:ten

In [84]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, shuffle=False, num_epochs=1, batch_size=len(X_test))
                                       

In [85]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Restoring parameters from C:\Users\RANBEE~1\AppData\Local\Temp\tmpnmp6wq30\model.ckpt-1885


In [86]:
final_pred = []
for pred in predictions:
    final_pred.append(pred['predictions'][0])

In [87]:
from sklearn.metrics import mean_squared_error
if t == 1:
    X_test['kda_ratio'] = final_pred
    X_test.to_csv(".sub.csv")
else:
    print(mean_squared_error(y_test, final_pred)**0.5)

572.739195397
